## Recombining Gun Violence Dataset 

The gunviolence_2013-2018.csv was too large to push to GitHub so the file was split into two smaller files.  This code recombines them.

This code also determines the category of the incident, i.e. mass shooting, domestic violence, police involved shooting.  Also since the data for 2018 was only through March, 2018 was excluded.


## Transforming Data for Mongo DB

This notebook loads csv files of data previously cleaned and loads them to Mongo DB and transforms them to JSON.

* cities - information on the income and poverty levels by city with associated lat and long
* states - infomation at the state level on income and state laws
* guns - information on the gun violence
---


In [1]:
import pandas as pd 
import os
import numpy as np
import datetime 
import csv

import pymongo
import json

import requests


# function to save dataframe to collection_name in MongoDB 'wines'    
def saveMongo(df, collection_name, replace=False):
    mng_client = pymongo.MongoClient('localhost', 27017)
    mng_db = mng_client['guns'] 
    if replace:
        mng_db[collection_name].drop() 
    db_cm = mng_db[collection_name]
    data = df
    data_json = json.loads(data.to_json(orient='records', date_unit='ns'))
    #db_cm.delete_many()
    db_cm.insert_many(data_json)
    


In [2]:
# Load CSV file
csv_path = os.path.join('..',"rawdata", "gun-violence-data_01-2013_12-2015.csv")

# Read the first half of the gun violence file and store into Pandas data frame
gun_violence_df_2015 = pd.read_csv(csv_path, encoding = "ISO-8859-1")
gun_violence_df_2015.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,1/1/13,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,1/1/13,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,1/1/13,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,1/5/13,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,1/7/13,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [3]:
# Load CSV file
csv_path = os.path.join('..',"rawdata", "gun-violence-data_01-2016_03-2018.csv")

# Read the second half of the gun violence file and store into Pandas data frame
gun_violence_df_2018 = pd.read_csv(csv_path, encoding = "ISO-8859-1")
gun_violence_df_2018.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,477000,1/1/16,Massachusetts,Boston,Fairmount Street,0,1,http://www.gunviolencearchive.org/incident/477000,http://www.bostonherald.com/news/local_coverag...,False,...,0::25,0::Adult 18+,0::Female,NaN,NaN,0::Injured,0::Victim,http://www.bostonherald.com/news/local_coverag...,NaN,NaN
1,1023080,1/1/16,Connecticut,Meriden,Hobart St and Myrtle St,0,0,http://www.gunviolencearchive.org/incident/102...,https://www.justice.gov/usao-ct/pr/west-haven-...,False,...,0::28,0::Adult 18+||1::Adult 18+||2::Adult 18+,0::Male||1::Male||2::Male,0::Don Meeker||1::Elbert Llorens||2::Kyle Vale...,NaN,"0::Unharmed, Arrested||1::Unharmed, Arrested||...",0::Subject-Suspect||1::Subject-Suspect||2::Sub...,https://www.justice.gov/usao-ct/pr/west-haven-...,84.0,13.0
2,476977,1/1/16,Missouri,Kansas City,12th Street and Denver Avenue,0,1,http://www.gunviolencearchive.org/incident/476977,http://www.kmbc.com/news/6-people-struck-by-gu...,False,...,NaN,NaN,0::Male,NaN,NaN,0::Injured,0::Victim,http://www.kmbc.com/news/6-people-struck-by-gu...,23.0,9.0
3,476979,1/1/16,Missouri,Kansas City,39th Street and U.S. 71,0,1,http://www.gunviolencearchive.org/incident/476979,http://www.kmbc.com/news/6-people-struck-by-gu...,False,...,NaN,NaN,0::Male,NaN,NaN,0::Injured,0::Victim,http://www.kmbc.com/news/6-people-struck-by-gu...,18.0,17.0
4,476661,1/1/16,Wisconsin,Cudahy,3700 block of E. Whittaker Ave,2,0,http://www.gunviolencearchive.org/incident/476661,http://hispanicnewsnetwork.blogspot.com/2016/0...,False,...,0::2||1::27,0::Child 0-11||1::Adult 18+,0::Female||1::Male,0::Skylar Monroe Hartman||1::Brett Hartman,1::Family,0::Killed||1::Killed,0::Victim||1::Subject-Suspect,http://www.jsonline.com/news/crime/father-kill...,20.0,7.0


In [4]:
# Recomine the two files
gun_violence_df= pd.concat([gun_violence_df_2015, gun_violence_df_2018])
gun_violence_df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,1/1/13,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,1/1/13,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,1/1/13,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,1/5/13,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,1/7/13,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [5]:
# Convert the date field to date/time and removed unnecessary columns
gun_violence_df['date']= pd.to_datetime(gun_violence_df['date']) 
gun_violence_df=gun_violence_df.loc[(gun_violence_df['date'] <'2018-01-01') & (gun_violence_df['date']>'2013-12-31') ]
gun_violence_df.drop(columns=['address', 'incident_url', 'incident_url_fields_missing', 'source_url', 'participant_name','sources', 'location_description','notes'], inplace=True, axis=1)
gun_violence_df.head()

,incident_id,date,state,city_or_county,n_killed,n_injured,congressional_district,gun_stolen,gun_type,incident_characteristics,...,longitude,n_guns_involved,participant_age,participant_age_group,participant_gender,participant_relationship,participant_status,participant_type,state_house_district,state_senate_district
278,95289,2014-01-01,Michigan,Muskegon,0,0,2.0,NaN,NaN,Shots Fired - No Injuries,...,-86.2514,NaN,NaN,0::Adult 18+,0::Female,NaN,0::Unharmed,0::Victim,92.0,34.0
279,92401,2014-01-01,New Jersey,Newark,0,0,10.0,NaN,NaN,Officer Involved Incident,...,-74.1695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,29.0
280,92383,2014-01-01,New York,Queens,1,0,5.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)",...,-73.7474,NaN,0::22||1::26,0::Adult 18+||1::Adult 18+,0::Male||1::Male,NaN,0::Killed||1::Unharmed,0::Victim||1::Subject-Suspect,33.0,14.0
281,92142,2014-01-01,New York,Brooklyn,0,1,9.0,NaN,NaN,Shot - Wounded/Injured,...,-73.9476,NaN,0::34,0::Adult 18+||1::Adult 18+,0::Male||1::Male,NaN,0::Injured,0::Victim||1::Subject-Suspect,43.0,20.0
282,95261,2014-01-01,Missouri,Springfield,0,1,7.0,NaN,NaN,Shot - Wounded/Injured,...,-93.3007,NaN,0::6||1::12,0::Child 0-11||1::Teen 12-17,0::Female,NaN,0::Injured||1::Unharmed,0::Victim||1::Subject-Suspect,131.0,30.0


In [6]:
# Search the incident_characteristics for specific incident types and set that incident type to True
gun_violence_df["mass"]=np.where(gun_violence_df['incident_characteristics'].str.contains("Mass Shooting", case=False, na=False), True, False)
gun_violence_df["gang"]=np.where(gun_violence_df['incident_characteristics'].str.contains("Gang", case=False, na=False), True, False)
gun_violence_df["domestic"]=np.where(gun_violence_df['incident_characteristics'].str.contains("Domestic Violence", case=False, na=False), True, False)
gun_violence_df["non-shooting"]=np.where(gun_violence_df['incident_characteristics'].str.contains("Non-Shooting", case=False, na=False), True, False)
gun_violence_df["accidental"]=np.where(gun_violence_df['incident_characteristics'].str.contains("Accidental", case=False, na=False), True, False)
gun_violence_df["prohibited"]=np.where(gun_violence_df['incident_characteristics'].str.contains("prohibited", case=False, na=False), True, False)
gun_violence_df['officer'] = np.where(gun_violence_df['incident_characteristics'].str.contains("Officer|TSA", case=False, na=False), True, False)
gun_violence_df.head()

,incident_id,date,state,city_or_county,n_killed,n_injured,congressional_district,gun_stolen,gun_type,incident_characteristics,...,participant_type,state_house_district,state_senate_district,mass,gang,domestic,non-shooting,accidental,prohibited,officer
278,95289,2014-01-01,Michigan,Muskegon,0,0,2.0,NaN,NaN,Shots Fired - No Injuries,...,0::Victim,92.0,34.0,False,False,False,False,False,False,False
279,92401,2014-01-01,New Jersey,Newark,0,0,10.0,NaN,NaN,Officer Involved Incident,...,NaN,29.0,29.0,False,False,False,False,False,False,True
280,92383,2014-01-01,New York,Queens,1,0,5.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)",...,0::Victim||1::Subject-Suspect,33.0,14.0,False,False,False,False,True,False,False
281,92142,2014-01-01,New York,Brooklyn,0,1,9.0,NaN,NaN,Shot - Wounded/Injured,...,0::Victim||1::Subject-Suspect,43.0,20.0,False,False,False,False,False,False,False
282,95261,2014-01-01,Missouri,Springfield,0,1,7.0,NaN,NaN,Shot - Wounded/Injured,...,0::Victim||1::Subject-Suspect,131.0,30.0,False,False,False,False,False,False,False


## Load csv files into pandas dataframes, clean, save to mongo db

In [7]:
# read in cities data
cities_path = os.path.join("..","Data","Cities.csv")

df_cities = pd.read_csv(cities_path, encoding="UTF-8")
df_cities.head()

# # save to/replace collection "cities" in "guns" mongo db
saveMongo(df_cities, "cities", replace=True)


In [8]:
# read in state data
states_path = os.path.join("..","Data","States.csv")

df_states = pd.read_csv(states_path, encoding="UTF-8")
df_states = df_states[["state","census_2010","pop_estimate_2015","2015_median_income", "age18longgunpossess","age21longgunpossess","assault","mentalhealth","universal"]]

df_states.head()


# # save to/replace collection "states" in "guns" mongo db
saveMongo(df_states, "states", replace=True)

In [12]:
# Loading gun violence
df_guns = gun_violence_df

df_guns = df_guns[["incident_id","date","state","city_or_county","n_killed","n_injured","incident_characteristics","latitude","longitude","mass","gang","domestic","non-shooting","accidental","prohibited","officer"]]
df_guns["n_involved"] = df_guns["n_killed"]+df_guns["n_injured"]
df_guns["year"]= pd.DatetimeIndex(df_guns['date']).year

# Create a column to record type of shooting
conditions = [
    (df_guns["mass"]==1),
    (df_guns["n_involved"] == 0),
    (df_guns["n_killed"]==0)]
choices = ["mass shooting", "no injuries","injuries only"]
df_guns["shoot_type"] = np.select(conditions, choices, default="some dead")

df_guns.head()

# Add in state level data for filtering purposes
df_guns_complete = pd.merge(df_guns, df_states, on="state", how="left")
df_guns_complete["count"] = 1
df_guns_complete.head()

# save to/replace collection "guns" in "guns" mongo db
saveMongo(df_guns_complete, "guns", replace=True)


/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

,incident_id,date,state,city_or_county,n_killed,n_injured,incident_characteristics,latitude,longitude,mass,...,shoot_type,census_2010,pop_estimate_2015,2015_median_income,age18longgunpossess,age21longgunpossess,assault,mentalhealth,universal,count
0,95289,2014-01-01,Michigan,Muskegon,0,0,Shots Fired - No Injuries,43.2301,-86.2514,False,...,no injuries,9883640,9932573,54203,1.0,0.0,0.0,0.0,0.0,1
1,92401,2014-01-01,New Jersey,Newark,0,0,Officer Involved Incident,40.7417,-74.1695,False,...,no injuries,8791894,8870869,68357,1.0,0.0,1.0,0.0,0.0,1
2,92383,2014-01-01,New York,Queens,1,0,"Shot - Dead (murder, accidental, suicide)",40.7034,-73.7474,False,...,some dead,19378102,19661411,58005,0.0,0.0,1.0,1.0,1.0,1
3,92142,2014-01-01,New York,Brooklyn,0,1,Shot - Wounded/Injured,40.6715,-73.9476,False,...,injuries only,19378102,19661411,58005,0.0,0.0,1.0,1.0,1.0,1
4,95261,2014-01-01,Missouri,Springfield,0,1,Shot - Wounded/Injured,37.2646,-93.3007,False,...,injuries only,5988927,6071745,59196,0.0,0.0,0.0,0.0,0.0,1


In [10]:
summary_guns_df = df_guns_complete.groupby("shoot_type",as_index=False).sum()[["pop_estimate_2015"]]
summary_guns_df["shoot_type"] = df_guns_complete.groupby("shoot_type",as_index=False).first()["shoot_type"]
summary_guns_df["Count"] = df_guns_complete.groupby("shoot_type",as_index=False).sum()[["count"]]
summary_guns_df["n_killed"]= df_guns_complete.groupby("shoot_type",as_index=False).sum()[["n_killed"]]
summary_guns_df["Incidents_per_100M"] = summary_guns_df ["Count"]/summary_guns_df["pop_estimate_2015"]*100000000 
summary_guns_df["Killed_per_100M"] = summary_guns_df ["n_killed"]/summary_guns_df["pop_estimate_2015"]*100000000 
summary_guns_df.reset_index()
summary_guns_df.head()

# save to/replace collection "guns_summary" in "guns" mongo db
saveMongo(summary_guns_df, "guns_summary", replace=True)

In [17]:
summary_states_df = df_guns_complete.groupby(["shoot_type","state"], as_index=False).sum()[["pop_estimate_2015"]]
summary_states_df["state"]= df_guns_complete.groupby(["shoot_type", "state"],as_index=False).first()["state"]
summary_states_df["shoot_type"] = df_guns_complete.groupby(["shoot_type", "state"],as_index=False).first()["shoot_type"]
summary_states_df["Count"] = df_guns_complete.groupby(["shoot_type", "state"],as_index=False).sum()[["count"]]
summary_states_df["n_killed"]= df_guns_complete.groupby(["shoot_type","state"],as_index=False).sum()[["n_killed"]]
summary_states_df["Incidents_per_100M"] = summary_states_df ["Count"]/summary_states_df["pop_estimate_2015"]*100000000 
summary_states_df["Killed_per_100M"] = summary_states_df ["n_killed"]/summary_states_df["pop_estimate_2015"]*100000000 
summary_states_df["2015_median_income"]= df_guns_complete.groupby(["shoot_type", "state"],as_index=False).first()["2015_median_income"]
summary_states_df["age18longgunpossess"]= df_guns_complete.groupby(["shoot_type", "state"],as_index=False).first()["age18longgunpossess"]
summary_states_df["age21longgunpossess"]= df_guns_complete.groupby(["shoot_type", "state"],as_index=False).first()["age21longgunpossess"]
summary_states_df["assault"]= df_guns_complete.groupby(["shoot_type", "state"],as_index=False).first()["assault"]
summary_states_df["mentalhealth"]= df_guns_complete.groupby(["shoot_type", "state"],as_index=False).first()["mentalhealth"]
summary_states_df["universal"]= df_guns_complete.groupby(["shoot_type", "state"],as_index=False).first()["universal"]

summary_states_df.reset_index()
summary_states_df.head()

# save to/replace collection "state_summary" in "guns" mongo db
saveMongo(summary_states_df, "state_summary", replace=True)
